In [12]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [61]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

### Use pandas to import and load into a dataframe

In [62]:
URL = 'https://storage.googleapis.com/applied-dl/heart.csv'
dataframe = pd.read_csv(URL)
dataframe.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


#### Split dataframe into training, validation and test sets.

In [63]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

193 train examples
49 validation examples
61 test examples


#### Input pipeline

In [71]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

##### using small bactch size for demonstration

In [72]:
batch_size = 5 
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

##### Printing pipeline to see format

In [73]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of ages:', feature_batch['age'])
  print('A batch of targets:', label_batch )

Every feature: ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']
A batch of ages: tf.Tensor([59 60 56 45 43], shape=(5,), dtype=int32)
A batch of targets: tf.Tensor([0 0 1 0 0], shape=(5,), dtype=int32)


##### Using the small batch to demo several types of feature columns

In [74]:
example_batch = next(iter(train_ds))[0]

##### Util func for creating feature column and transform batch of data

In [75]:
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

##### Numeric column.  Simplest.  Represents real valued features.  Recives value from dataframe unchanged

In [76]:
age = feature_column.numeric_column("age")
demo(age)

[[59.]
 [60.]
 [56.]
 [45.]
 [43.]]


##### Bucketized column.

In [77]:
age_buckets = feature_column.bucketized_column(age, boundaries=[18,25,30,35,40,45,50,55,60,65])
demo(age_buckets)

[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


##### Categorical column.  "thal" is represented as a string.  We need to map it to a numerical value in order to pass it to the model. 

In [78]:
thal = feature_column.categorical_column_with_vocabulary_list(
    'thal', ['fixed', 'normal', 'reversible'])

thal_one_hot = feature_column.indicator_column(thal)
demo(thal_one_hot)

[[0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


##### Embedding.  Can't use one-hot on large numbers of categories.  Embedding is good in this situation.

In [79]:
#input is from the categorical column from above ^^^
thal_embedding = feature_column.embedding_column(thal, dimension=8)
demo(thal_embedding)

[[ 0.5227769  -0.45963043 -0.19177355  0.15984991 -0.24282959  0.0995312
   0.14461643  0.4610218 ]
 [ 0.5227769  -0.45963043 -0.19177355  0.15984991 -0.24282959  0.0995312
   0.14461643  0.4610218 ]
 [ 0.34571877  0.09422885  0.1663026  -0.38047174 -0.2872325   0.17225757
  -0.29787958 -0.30756164]
 [ 0.5227769  -0.45963043 -0.19177355  0.15984991 -0.24282959  0.0995312
   0.14461643  0.4610218 ]
 [ 0.5227769  -0.45963043 -0.19177355  0.15984991 -0.24282959  0.0995312
   0.14461643  0.4610218 ]]


##### Hashed column 

In [80]:
#Downside can be collision from diff strings mapped to same bucket. Can still work well regardless.
#Make bucket size well larger than num of categories.
thal_hashed = feature_column.categorical_column_with_hash_bucket(
    'thal', hash_bucket_size=1000)
demo(feature_column.indicator_column(thal_hashed))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


##### Feature crosses.  Combine features into a single feature.  Model can learn separate weights for each combo.

###### Getting error. Python int too large to convert to C long. 

In [102]:
#crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
#demo(feature_column.indicator_column(crossed_feature))
#import sys
#print(sys.maxsize)

In [94]:
feature_columns = []

# numeric cols
for header in ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'slope', 'ca']:
  feature_columns.append(feature_column.numeric_column(header))

# bucketized cols
age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
feature_columns.append(age_buckets)

# indicator cols
thal = feature_column.categorical_column_with_vocabulary_list(
      'thal', ['fixed', 'normal', 'reversible'])
thal_one_hot = feature_column.indicator_column(thal)
feature_columns.append(thal_one_hot)

# embedding cols
thal_embedding = feature_column.embedding_column(thal, dimension=8)
feature_columns.append(thal_embedding)

# crossed cols
#crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
#crossed_feature = feature_column.indicator_column(crossed_feature)
#feature_columns.append(crossed_feature)

In [95]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [96]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [97]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=5)

Epoch 1/5
7/7 [==============================] - 0s 52ms/step - loss: 3.2777 - accuracy: 0.4593 - val_loss: 2.5571 - val_accuracy: 0.6939
Epoch 2/5
7/7 [==============================] - 0s 23ms/step - loss: 2.4148 - accuracy: 0.6791 - val_loss: 0.6049 - val_accuracy: 0.6939
Epoch 3/5
7/7 [==============================] - 0s 23ms/step - loss: 0.8598 - accuracy: 0.7181 - val_loss: 0.7450 - val_accuracy: 0.6939
Epoch 4/5
7/7 [==============================] - 0s 22ms/step - loss: 0.6007 - accuracy: 0.7375 - val_loss: 0.8647 - val_accuracy: 0.7347
Epoch 5/5
7/7 [==============================] - 0s 23ms/step - loss: 0.7136 - accuracy: 0.7347 - val_loss: 0.7922 - val_accuracy: 0.7347


In [98]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

2/2 [==============================] - 0s 12ms/step - loss: 0.7513 - accuracy: 0.7213
Accuracy 0.72131145


###### 